In [1]:
import numpy as np
seed = 1
np.random.seed(seed)

In [2]:
### Sigmoid
def sigmoid(Z):
    
    Z[Z > 709] =  709 #prevent np.exp overflow
    Z[Z <-709] =  0   #prevent np.exp overflow
    
    A = np.where(Z >= 0, 
                    1. / (1. + np.exp(-Z)), 
                    np.exp(Z) / (1. + np.exp(Z))) # Activation
    C = Z                       # Cache
    
    return A , C 


def sigmoid_prime(dA, C): ## Derivative and Cache
    
    Z = C                 ## Cache
    
    S = 1. / (1. +np.exp(-Z))
    ds = S * (1-S)
    
    dZ = dA * ds
    
    return dZ

def sigmoid_backward(dA, cache):
    
    Z = cache
    
    s = 1/(1+np.exp(-Z))
    dZ = dA * s * (1-s)
    
    assert (dZ.shape == Z.shape)
    
    return dZ


### RELU
def relu(Z, alpha=0.0000000001):
    C = Z                       # Cache
    A = np.maximum(alpha*Z,Z)     # Activation
    
    return A , C 


def relu_prime(dA, C):
    
    Z = C                      ## Cache
    
    dZ = np.array(dA, copy=True)
    
    dZ[Z <= 0] = 0
    #dZ = dZ * 2
    
    return dZ


### tanH
def tanh(Z):
    
    C = Z                       # Cache
    A = np.tanh(Z)              # Activation
    
    return A , C


def tanh_prime(dA, C):
    
    Z = C                      ## Cache
    
    dZ = 1-np.tanh(dA)**2
    
    return dZ

### Identity
def identity(Z):
    
    C = Z                       # Cache
    A = Z                       # Activation
    
    return A , C


def identity_prime(dA, C):
    
    Z = C                      ## Cache
    
    dZ = dA
    
    return dZ


### SoftMax
def softmax(Z, axis=1):
    
    C = Z
    A = np.exp(x) / np.sum(np.exp(x), axis = axis, keepdims = True)
    
    return A , C

def softmax_prime(dA, C):
    
    Z = C                      ## Cache
    
    s = dA.reshape(-1,1)
    dZ = p.diagflat(s) - np.dot(s, s.T)
    
    return dZ

In [3]:
### Loss L1
def L1(yhat, y):
    
    loss = np.sum(abs(y-yhat))
    
    return loss


### Loss L2
def L2(yhat, y):
    
    loss = np.dot( (y-yhat),(y-yhat) )
    
    return loss


### Loss Mean Square Error
def mse(yhat, y):
    
    loss = np.mean(np.power(y-yhat, 2))
    
    return loss


def mse_prime(yhat, y):
    
    loss_prime = (2 * (y-yhat)) /y.size
    
    return loss_prime


### Loss cross entopy cost
def cross_entropy_cost(yhat, y):
    
    m = y.shape[1]
    
    cost = np.squeeze( (1./m) * (-np.dot(y,np.log(yhat).T) - np.dot(1-y, np.log(1-yhat).T)) )
    
    return cost

### Loss cross entopy cost
def cross_entropy_cost_prime(yhat, y):
    
    dA = - (np.divide(y, yhat) - np.divide(1 - y, 1 - yhat))
    
    return dA


In [4]:
activation_map = {
        'Identity':(identity,identity_prime),
        'Sigmoid' :(sigmoid ,sigmoid_prime ),
        'Relu'    :(relu    ,relu_prime    ),
        'Tanh'    :(tanh    ,tanh_prime    ),
        'Softmax' :(softmax ,softmax_prime )
    }

In [5]:
class NN_Unit:
    def __init__(self, activation_fn='Identity'):
        
        self.n_X = None  # Input_size
        self.n_A = None  # Output_size
        
        self.activation_f = activation_map[activation_fn][0]
        self.activation_b = activation_map[activation_fn][1]
        
    def forward(self, X):
        raise NotImplementedError
        
    def backward(self, dA, cache):
        raise NotImplementedError
        
    def init(self):
        raise NotImplementedError
        

In [6]:
# from nn_unit import NN_Unit

In [7]:
class FullyConnectedUnit(NN_Unit):
    
    def __init__(self, n_X, n_A, activation_fn): # num Input , num_output
        super(FullyConnectedUnit, self).__init__(activation_fn)
        
        self.n_X = n_X
        self.n_A = n_A
        
        self.W = np.random.randn(n_A, n_X) * 0.01
        self.B = np.zeros((n_A, 1))        
                
    def forward(self, X):
        
        self.X = X
        
        self.Z = self.W.dot(self.X) + self.B
        
        Z_cache = (self.X, self.W, self.B)
        
        self.A, A_cache = self.activation_f(self.Z)
        return self.A, (Z_cache, A_cache)

    def backward(self, dA, cache):
        
        Z_cache, A_cache = cache
        
        dZ = self.activation_b(dA, A_cache)
        
        A , W , B = Z_cache        
        
        m = A.shape[1]
        
        dW = 1./m * np.dot(dZ,A.T)
        dB = 1./m * np.sum(dZ, axis = 1, keepdims = True)
        dA = np.dot(W.T,dZ)
        
        return dA, dW, dB
    
    def init(self):
        
        np.random.seed(seed)
        self.W = np.random.randn(self.n_A, self.n_X) / np.sqrt(self.n_X)
        self.B = np.zeros((self.n_A, 1))
    

In [8]:
class MaxPool2dUnit(NN_Unit):
    
    # n_A is here pool_filter_size
    def __init__(self, n_X, n_Y, n_A, stride=2, activation_fn='Identity'): # num Input , num_output
        super(FullyConnectedUnit, self).__init__(activation_fn)
        
        self.n_X = n_X
        self.n_X = n_Y
        self.n_A = n_A
        
    def forward(self, X):
        
        pass
    
    def backward(self, dA, cache):
        
        pass
    
    def init(self):
        
        pass
    

In [9]:
class NN_Layer:
    def __init__(self):
        self.nn_unit = None
        
    def forward(self, X):
        raise NotImplementedError
        
    def backward(self, dA, cache):
        raise NotImplementedError
        
    def update(self, grads, lr=0.001):
        raise NotImplementedError
        
    def init(self):
        raise NotImplementedError

In [10]:
# from layer import NN_Layer

In [11]:
class FullyConnectedLayer(NN_Layer):
    
    def __init__(self, num_in, num_out, activation_fn): # num Input , num_output
        
        self.ins  = num_in
        self.outs = num_out
        self.nn_unit = FullyConnectedUnit(num_in,num_out,activation_fn)
                
    def forward(self, X):
        
        self.A, (Z_cache, A_cache) = self.nn_unit.forward(X)
        
        return self.A, (Z_cache, A_cache)

    def backward(self, dA, cache):
        
        dA, dW, dB = self.nn_unit.backward(dA, cache)
        
        return dA, dW, dB
    
    def update(self, dW, dB, lr=0.01):
        self.nn_unit.W = self.nn_unit.W - (lr * dW)
        self.nn_unit.B = self.nn_unit.B - (lr * dB)
        
    def init(self):
        self.nn_unit.init()

In [12]:
def load_data():
    train_dataset = h5py.File('datasets/train_catvnoncat.h5', "r")
    train_set_x_orig = np.array(train_dataset["train_set_x"][:]) # your train set features
    train_set_y_orig = np.array(train_dataset["train_set_y"][:]) # your train set labels

    test_dataset = h5py.File('datasets/test_catvnoncat.h5', "r")
    test_set_x_orig = np.array(test_dataset["test_set_x"][:]) # your test set features
    test_set_y_orig = np.array(test_dataset["test_set_y"][:]) # your test set labels

    classes = np.array(test_dataset["list_classes"][:]) # the list of classes
    
    train_set_y_orig = train_set_y_orig.reshape((1, train_set_y_orig.shape[0]))
    test_set_y_orig = test_set_y_orig.reshape((1, test_set_y_orig.shape[0]))
    
    return train_set_x_orig, train_set_y_orig, test_set_x_orig, test_set_y_orig, classes

import h5py
train_x_orig, train_y, test_x_orig, test_y, classes = load_data()

train_x_flatten = train_x_orig.reshape(train_x_orig.shape[0], -1).T   # The "-1" makes reshape flatten the remaining dimensions
test_x_flatten = test_x_orig.reshape(test_x_orig.shape[0], -1).T

# Standardize data to have feature values between 0 and 1.
train_x = train_x_flatten/255.
test_x = test_x_flatten/255.

In [13]:
class NN_Network:

    def __init__(self, l_rate=0.01):
        
        self.layers = []
        self.l_rate = l_rate
        
    def add_layer(self, layer):
        num_out, num_in = layer.outs, layer.ins
        self.layers.append(layer)
        layer.init()
                
    def train(self, X, Y, epochs= 100):
        for e in range(0,epochs):
            A = X
            caches = []
            for idx, l in enumerate(self.layers):
                A, cache = l.forward(A)
                caches.append(cache)
            
            cost = cross_entropy_cost(A, Y)
            
            dA = cross_entropy_cost_prime(A, Y)
                
            cnt = len(self.layers)
            Y = Y.reshape(A.shape)
            
            for i in reversed(range(0, cnt)):
                dA, dW, dB = self.layers[i].backward(dA, caches[i])
                self.layers[i].update(dW, dB)
            

            if e%100 == 0 and e != 0:
                print(f"Cost at epoch {e} is {cost}")
            
    def predict(self, X):
        A = X
        for idx, l in enumerate(self.layers):
            A, cache = l.forward(A)
        
        A[A>=0.5]= 1
        A[A< 0.5]= 0
        
        return A
    
    def evaluate(self, X, Y):
        Yhat = self.predict(X).astype(int)
        print("Accuracy: "  + str(np.sum((Yhat.astype(int) == Y)/Y.shape[1])))
        return Yhat

In [14]:
fc0 = FullyConnectedLayer(12288,512,'Relu')
fc1 = FullyConnectedLayer(512,16,'Relu')
fcf = FullyConnectedLayer(16,1,'Sigmoid')

In [15]:
model = NN_Network()

In [16]:
model.add_layer(fc0)
model.add_layer(fc1)
model.add_layer(fcf)

In [17]:
model.train(train_x, train_y, 2500)

Cost at epoch 100 is 0.49238334060411304
Cost at epoch 200 is 0.39120726086224006
Cost at epoch 300 is 0.3016951246630109
Cost at epoch 400 is 0.20489188247015286
Cost at epoch 500 is 0.12428382078720963
Cost at epoch 600 is 0.07062776707009956
Cost at epoch 700 is 0.04327363162376949
Cost at epoch 800 is 0.030559793125277006
Cost at epoch 900 is 0.022929424092737596
Cost at epoch 1000 is 0.018032471207089333
Cost at epoch 1100 is 0.014628058818493076
Cost at epoch 1200 is 0.012147554720710663
Cost at epoch 1300 is 0.01027773085121551
Cost at epoch 1400 is 0.008837663966558466
Cost at epoch 1500 is 0.0077122095565339225
Cost at epoch 1600 is 0.006803003702345492
Cost at epoch 1700 is 0.0060584105745395325
Cost at epoch 1800 is 0.005441119540565534
Cost at epoch 1900 is 0.0049267002194969284
Cost at epoch 2000 is 0.004486548090775295
Cost at epoch 2100 is 0.004110198636251039
Cost at epoch 2200 is 0.0037838031502715735
Cost at epoch 2300 is 0.0035006255539303736
Cost at epoch 2400 is 0.

In [18]:
probs = model.evaluate(test_x, test_y)
probs

Accuracy: 0.76


array([[1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0,
        0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0,
        1, 0, 0, 1, 1, 0]])

In [19]:
test_y

array([[1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0,
        0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0,
        0, 0, 1, 1, 1, 0]], dtype=int64)

In [20]:
model.predict(test_x)

array([[1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 0., 1.,
        0., 1., 0., 0., 1., 0., 0., 1., 1., 1., 0., 0., 0., 1., 1., 1.,
        1., 1., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 1., 0., 0., 1.,
        1., 0.]])